In [9]:
from abc import ABC, abstractmethod

# Mediator Interface
class DialogMediator(ABC):
    @abstractmethod
    def notify(self, sender, event):
        pass

# Concrete Mediator
class AuthenticationDialog(DialogMediator):
    def __init__(self):
        self.username_field = UsernameField(self)
        self.password_field = PasswordField(self)
        self.login_button = LoginButton(self)
        self.message_label = MessageLabel(self)

    def notify(self, sender, event):
        if sender == self.login_button and event == "click":
            username = self.username_field.get_text()
            password = self.password_field.get_text()
            if self.authenticate(username, password):
                self.message_label.set_text("Login successful!")
            else:
                self.message_label.set_text("Invalid credentials.")

    def authenticate(self, username, password):
        # Simulate authentication logic
        return username == "admin" and password == "password"

# Colleague Components
class Component(ABC):
    def __init__(self, mediator):
        self.mediator = mediator

class UsernameField(Component):
    def __init__(self, mediator):
        super().__init__(mediator)
        self.text = ""

    def set_text(self, text):
        self.text = text

    def get_text(self):
        return self.text

class PasswordField(Component):
    def __init__(self, mediator):
        super().__init__(mediator)
        self.text = ""

    def set_text(self, text):
        self.text = text

    def get_text(self):
        return self.text

class LoginButton(Component):
    def click(self):
        self.mediator.notify(self, "click")

class MessageLabel(Component):
    def set_text(self, text):
        print(f"Message: {text}")



In [10]:
# Client Code
def main():
    dialog = AuthenticationDialog()

    # Simulate user input
    dialog.username_field.set_text("admin")
    dialog.password_field.set_text("password")

    # Simulate button click
    dialog.login_button.click()

    # Simulate incorrect login
    dialog.username_field.set_text("user")
    dialog.password_field.set_text("1234")
    dialog.login_button.click()

if __name__ == "__main__":
    main()

Message: Login successful!
Message: Invalid credentials.


In [11]:
from abc import ABC, abstractmethod
import datetime

# Mediator Interface
class ChatRoomMediator(ABC):
    @abstractmethod
    def show_message(self, sender, message):
        pass

    @abstractmethod
    def send_private_message(self, sender, receiver_name, message):
        pass

    @abstractmethod
    def register_user(self, user):
        pass

# Concrete Mediator
class ChatRoom(ChatRoomMediator):
    def __init__(self):
        self.users = {}

    def register_user(self, user):
        if user.name not in self.users:
            self.users[user.name] = user
            print(f"{user.name} has joined the chat room.")
        else:
            print(f"User with name {user.name} already exists.")

    def show_message(self, sender, message):
        time = datetime.datetime.now().strftime("%H:%M")
        print(f"[{time}] {sender.name}: {message}")
        for user in self.users.values():
            if user != sender and user.is_online:
                user.receive(message, sender.name)

    def send_private_message(self, sender, receiver_name, message):
        receiver = self.users.get(receiver_name)
        if receiver and receiver.is_online:
            time = datetime.datetime.now().strftime("%H:%M")
            print(f"[{time}] {sender.name} to {receiver.name} (private): {message}")
            receiver.receive_private(message, sender.name)
        else:
            print(f"{sender.name}, the user {receiver_name} is not available.")

# Colleague Abstract Class
class User(ABC):
    def __init__(self, name, mediator):
        self.name = name
        self.mediator = mediator
        self.is_online = True
        self.mediator.register_user(self)

    @abstractmethod
    def send(self, message):
        pass

    @abstractmethod
    def send_private(self, receiver_name, message):
        pass

    @abstractmethod
    def receive(self, message, sender_name):
        pass

    @abstractmethod
    def receive_private(self, message, sender_name):
        pass

    def go_offline(self):
        self.is_online = False
        print(f"{self.name} is now offline.")

    def go_online(self):
        self.is_online = True
        print(f"{self.name} is now online.")

# Concrete Colleague
class ConcreteUser(User):
    def send(self, message):
        print(f"{self.name} sends message: {message}")
        self.mediator.show_message(self, message)

    def send_private(self, receiver_name, message):
        print(f"{self.name} sends private message to {receiver_name}: {message}")
        self.mediator.send_private_message(self, receiver_name, message)

    def receive(self, message, sender_name):
        print(f"{self.name} received message from {sender_name}: {message}")

    def receive_private(self, message, sender_name):
        print(f"{self.name} received private message from {sender_name}: {message}")


In [12]:
# Client Code
def main():
    # Create mediator
    chat_room = ChatRoom()

    # Create users
    alice = ConcreteUser("Alice", chat_room)
    bob = ConcreteUser("Bob", chat_room)
    charlie = ConcreteUser("Charlie", chat_room)

    print("\n--- Public Messages ---")
    alice.send("Hello, everyone!")
    bob.send("Hi Alice!")
    charlie.send("Hey folks, what's up?")

    print("\n--- Private Messages ---")
    alice.send_private("Bob", "Are you joining the meeting later?")
    bob.send_private("Alice", "Yes, I'll be there.")

    print("\n--- User Goes Offline ---")
    charlie.go_offline()
    alice.send("Looks like Charlie went offline.")

    print("\n--- Messages After User Offline ---")
    bob.send("Charlie, are you there?")
    charlie.send("I'm back!")  # This message won't be sent because Charlie is offline

    print("\n--- User Comes Back Online ---")
    charlie.go_online()
    charlie.send("Sorry, I was away.")

if __name__ == "__main__":
    main()


Alice has joined the chat room.
Bob has joined the chat room.
Charlie has joined the chat room.

--- Public Messages ---
Alice sends message: Hello, everyone!
[16:59] Alice: Hello, everyone!
Bob received message from Alice: Hello, everyone!
Charlie received message from Alice: Hello, everyone!
Bob sends message: Hi Alice!
[16:59] Bob: Hi Alice!
Alice received message from Bob: Hi Alice!
Charlie received message from Bob: Hi Alice!
Charlie sends message: Hey folks, what's up?
[16:59] Charlie: Hey folks, what's up?
Alice received message from Charlie: Hey folks, what's up?
Bob received message from Charlie: Hey folks, what's up?

--- Private Messages ---
Alice sends private message to Bob: Are you joining the meeting later?
[16:59] Alice to Bob (private): Are you joining the meeting later?
Bob received private message from Alice: Are you joining the meeting later?
Bob sends private message to Alice: Yes, I'll be there.
[16:59] Bob to Alice (private): Yes, I'll be there.
Alice received pr

In [13]:
class Mediator:
    def send_message(self, msg, department):
        pass

class Department:
    def __init__(self, mediator=None):
        self.mediator = mediator

    def send_message(self, msg):
        self.mediator.send_message(msg, self)

    def receive_message(self, msg):
        pass

class HRDepartment(Department):
    def receive_message(self, msg):
        print(f"HR Department received: {msg}")

class FinanceDepartment(Department):
    def receive_message(self, msg):
        print(f"Finance Department received: {msg}")

class OfficeMediator(Mediator):
    def __init__(self):
        self.hr = None
        self.finance = None

    def set_hr(self, hr):
        self.hr = hr

    def set_finance(self, finance):
        self.finance = finance

    def send_message(self, msg, department):
        if department == self.hr:
            if self.finance:
                self.finance.receive_message(msg)
        elif department == self.finance:
            if self.hr:
                self.hr.receive_message(msg)

# Test the implementation
mediator = OfficeMediator()
hr = HRDepartment(mediator)
finance = FinanceDepartment(mediator)

mediator.set_hr(hr)
mediator.set_finance(finance)

hr.send_message("HR update for all departments")


Finance Department received: HR update for all departments
